# Predicting Best Picture Winners & Nominees
*An Analysis by Sean Osier*

### Data Scraping with Selenium

In [117]:
# Import Dependencies
import pickle
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time

In [3]:
# Pickling functions
def pickle_it(data, filename, python_version=3):
    with open(filename, "wb") as picklefile:
        pickle.dump(data, picklefile, protocol=python_version)

def load_pickle(filename):
    with open(filename, "rb") as picklefile: 
        return pickle.load(picklefile)

In [106]:
# Selenium class and methods
class Selenium(object):
    def __init__(self, url):
        self.chromedriver = "/Users/seanosier/Downloads/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
        self.driver = webdriver.Chrome(chromedriver)
        self.driver.get(url)
        self.delay = 2  # seconds
        time.sleep(self.delay)
    
    def open_page(self, url):
        self.driver.get(url)
        time.sleep(self.delay) 

    def find(self, tag, attr, value):
        selector = "//%s[contains(@%s, '%s')]" % (tag, attr, value)
        return_list = []
        for item in self.driver.find_elements_by_xpath(selector):
            return_list.append(item.text)

        return return_list
        
    def close(self):
        self.driver.close()

In [107]:
# Selenium funcitons
def selenium_single_scrape(urls, tag, attr, value):
    sel = "Not yet initiated"
    all_results = []
    for i, url in enumerate(urls):
        if i == 0:
            sel = Selenium(url)
        else:
            sel.open_page(url)
        result_list = sel.find(tag, attr, value)
        all_results.append(result_list)
    sel.close()
    return tuple(all_results)

def scrape_google_frequently_mentioned_bar_names(urls):
    results = selenium_single_scrape(urls, "div", "class", "kltat")
    return results

In [112]:
def scrape_directors_actors_actresses():
    urls = ["https://www.google.com/#q=list%20of%20top%20directors",
            "https://www.google.com/#q=list%20of%20top%20actors",
            "https://www.google.com/#q=list%20of%20top%20actresses"]
    directors, actors, actresses = scrape_google_frequently_mentioned_bar_names(urls)
    return directors, actors, actresses

directors, actors, actresses = scrape_directors_actors_actresses()

In [111]:
# pickle_it(tuple([directors, actors, actresses]), "directors_actors_actresses.pkl")
!ls

Data_Processing.ipynb          detailed_movie_data4.pkl
Data_Scraping.ipynb            detailed_movie_data5.pkl
Selenium_Scraping.ipynb        detailed_movie_data6.pkl
all_links.pkl                  detailed_movie_data7.pkl
detailed_movie_data1.pkl       detailed_movie_data7b.pkl
detailed_movie_data10.pkl      detailed_movie_data8.pkl
detailed_movie_data11.pkl      detailed_movie_data9.pkl
detailed_movie_data12.pkl      directors_actors_actresses.pkl
detailed_movie_data13.pkl      movie_data.pkl
detailed_movie_data2.pkl       nominees_and_winners.pkl
detailed_movie_data3.pkl
